# M2E05: Python Implementations of Association Rule Mining

# Data Science Master 2021/22

# Prof. Dr. Gottfried Vossen

In this notebook we look at several Python implementations of association rule mining. We first look at two Apriori implementations and after that into frequent pattern growth.

## Contents:

1. [Association Rule Mining using apyori](#apyori)
2. [Using the Apriori implementation of mlxtend](#mlxtend)
3. [Shortcomings of Apriori](#shortc)
4. [Frequent Pattern Growth](#fpg)
5. [An Alternative FP-Growth implementation](#fpg2)
6. [Exercise: The LastFM Dataset](#lastfm)

<div style='align: left; text-align:center;'>
    <img src='img/associationrule.png' alt='Association rule mining' style="height:250px;"/>
    <span style='display:block;'>Association rule mining. Image Source: <a href="https://theneuralnetworkblog.wordpress.com/2020/05/06/association-rule-mining-apriori/" target="_blank">The Neural Network Blog</a>.</span>
    <br/>
</div>

## 1. Association Rule MIning using apyori <a class="anchor" id="apyori"/>

Our first Python implementation is described in detail in "Association Rule Mining via Apriori Algorithm in Python" by Usman Malik (see https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/).

## Load the data

We will apply the Apriori algorithm to a dataset of 7,500 transactions which have been recorded over the course of a week with a French retailer. To this end, the data is loaded into a pandas DataFrame which presents the data in tabular format and which can be derived from a csv or xsl file.

In [3]:
import os
import pandas as pd

store_data = pd.read_csv(os.path.join('Data-20220222', 'store_data.csv'), header=None)

In [4]:
store_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataset contains no column headers. In order to avoid that the first row is taken as header, we set "header=None".

In [5]:
store_data.info() # Show the number of rows and columns. Check memory usage as well.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       7501 non-null   object
 1   1       5747 non-null   object
 2   2       4389 non-null   object
 3   3       3345 non-null   object
 4   4       2529 non-null   object
 5   5       1864 non-null   object
 6   6       1369 non-null   object
 7   7       981 non-null    object
 8   8       654 non-null    object
 9   9       395 non-null    object
 10  10      256 non-null    object
 11  11      154 non-null    object
 12  12      87 non-null     object
 13  13      47 non-null     object
 14  14      25 non-null     object
 15  15      8 non-null      object
 16  16      4 non-null      object
 17  17      4 non-null      object
 18  18      3 non-null      object
 19  19      1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


## Apriori algorithm

[Apriori](https://en.wikipedia.org/wiki/Apriori_algorithm) identifies the frequent individual items in the dataset and extends them to larger and larger item sets as long as those item sets appear sufficiently often in the dataset. The frequent item sets determined by Apriori can be used to determine association rules which highlight general trends in the dataset, as in the following example:

<div style='align: left; text-align:center;'>
    <img src='img/exampleapriori.png' alt='Example of the Apriori algorithm' style="height:500px;"/>
    <span style='display:block;'>Example association rule mining using the Apriori algorithm. Image Source: <a href="https://developpaper.com/association-rule-mining-and-apriori-algorithm/" target="_blank">Developer Paper</a>.</span>
    <br/>
</div>

We will use the [apyori](https://github.com/ymoch/apyori) library developed by [Yu Mochizuki](https://github.com/ymoch/). Apyori offers a simple Python implementation of the Apriori algorithm and uses three measures:

1.    Support(B) = (Transactions containing (B))/(Total Transactions)

2.    Confidence(A→B) = (Transactions containing both (A and B))/(Transactions containing A)

3.    Lift(A→B) = (Confidence (A→B))/(Support (B))

where A and B are individual items.

To the be able to use the library and run the following code blocks, you first need to install apyori. Open your system's Terminial/Command Prompt and type:

```bash
pip install apyori
```

You will need to restart the Jupyter Notebook Kernel to apply the changes. You can do that by clicking on the Jupyter menu Kernel > Restart.

<div style="align: left; text-align:center;">
    <img src="img/restartkernel.png" alt="Restarting Notebook's Kernel" />
    <span style="display:block;">How to restart the Notebook's Kernel.</span>
    <br/>
</div>

After that, you should be able to use apyori by importing it. Eventually the environment in your Anaconda Navigator needs to be switched to "Anaconda 3".

You are now ready to execute the next code block.

The algorithm executes the following steps:

1. Set lower bounds for support and confidence.

2. Extract all item sets having support greater than the min-support.

3. Select all rules having confidence greater than the min-confidence.

(4. Order the rules according to descending lift.)

The Apriori library used here expects a dataset in the form of a list of lists, where the entire dataset is a big ("outer") list and each transaction within it is an "inner" list. Since the dataset is currently a pandas DataFrame, we convert it into the desired form:

In [13]:
import pip
pip.main(['install','apyori'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=2279e0cd2331cd795c4a3365f5175aed062f7c0a70cde797f8ee933c0b1002fd
  Stored in directory: /Users/ahmedtantawy/Library/Caches/pip/wheels/32/2a/54/10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apyori


0

In [14]:
from apyori import apriori
    
transactions = []

# Convert items in the dataframe to string
for row in store_data.values: 
    transactions.append(str(item) for item in row if str(item) != 'nan') # Remove missing values


Next we apply the Apriori algorithm to our dataset; to this end, we use the apriori class from the imported apyori library.

The apriori class expects varoius parameters in the following order:

1.    The list of lists from wich the rules are to be extracted.

2.    The min-support.

3.    The min-confidence.

4.    The min-lift.

5.    The minimum rule length as the minimum number of items in the rules that are generated.

Suppose we are interested in rules for only those items that are purchased at least 5 times a day, or 7 x 5 = 35 times in one week, since our dataset is for a one-week time period. The support for those items can be calculated as 35/7500 = 0.0045. The minimum confidence for the rules is 20% or 0.2. Similarly, we specify the value for lift as 3 and finally min_length is 2 since we want at least two products in our rules. These values are arbitrarily chosen and can be changed, so that you can see what difference it makes in the rules you get back out and get a feeling for which values yield interesting results and which do not..


In [15]:
# Run Apriori on top of the transactions
results = apriori(transactions, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)

association_rules = list(results) # Store rules in a list

In [16]:
for rule in association_rules:
    # Get items involved in the rule
    items =  [x for x in rule.items]
    print("Rule: " + items[0] + " -> " + items[1])
    
    # Print the support of the rule
    print("Support:", rule.support)
    
    # Print the confidence of the rule
    print("Confidence:", rule.ordered_statistics[0][2])
    
    # Print the lift of the rule
    print("Lift:", rule.ordered_statistics[0][3])
    
    # Print a separator
    print('---')

Rule: chicken -> light cream
Support: 0.004532728969470737
Confidence: 0.29059829059829057
Lift: 4.84395061728395
---
Rule: mushroom cream sauce -> escalope
Support: 0.005732568990801226
Confidence: 0.3006993006993007
Lift: 3.790832696715049
---
Rule: escalope -> pasta
Support: 0.005865884548726837
Confidence: 0.3728813559322034
Lift: 4.700811850163794
---
Rule: ground beef -> herb & pepper
Support: 0.015997866951073192
Confidence: 0.3234501347708895
Lift: 3.2919938411349285
---
Rule: ground beef -> tomato sauce
Support: 0.005332622317024397
Confidence: 0.3773584905660377
Lift: 3.840659481324083
---
Rule: whole wheat pasta -> olive oil
Support: 0.007998933475536596
Confidence: 0.2714932126696833
Lift: 4.122410097642296
---
Rule: shrimp -> pasta
Support: 0.005065991201173177
Confidence: 0.3220338983050847
Lift: 4.506672147735896
---
Rule: chocolate -> shrimp
Support: 0.005332622317024397
Confidence: 0.23255813953488375
Lift: 3.2545123221103784
---
Rule: ground beef -> spaghetti
Support:

Since the algorithm uses a lot of storage space, the original transaction list eaten up during an execution of the algorithm in the implementation used here, i.e., in the end the transaction list is empty and needs to be recreated for every further execution.

In [ ]:
transactions = []

# Convert items in the dataframe to string
for row in store_data.values:
    transactions.append(str(item) for item in row if str(item) != 'nan') # Remove missing values

In [ ]:
# Another example:

results = apriori(transactions, min_support=0.005, min_confidence=0.3, min_lift=4, min_length=2)

association_rules = list(results) # Store rules in a list

In [ ]:
for rule in association_rules:
    # Get items involved in the rule
    items =  [x for x in rule.items]
    print("Rule: " + items[0] + " -> " + items[1])
    
    # Print the support of the rule
    print("Support:", rule.support)
    
    # Print the confidence of the rule
    print("Confidence:", rule.ordered_statistics[0][2])
    
    # Print the lift of the rule
    print("Lift:", rule.ordered_statistics[0][3])
    
    # Print a separator
    print('---')

## 2. Using the Apriori implementation of mlxtend <a class="anchor" id="mlxtend"/>

As an alternative to `apyori`, we use in the follwing code blocks the [mlxtend](https://rasbt.github.io/mlxtend/) ("machine learning extensions") library. Follow the instructions below to install mlxtend.

**Installation**

To install mlxtend, just execute on Terminal/CommandPrompt:

```bash
pip install mlxtend
```

If you use Anaconda, it is recommended to use the following command:

```bash
conda install -c conda-forge mlxtend
```

**Don't forget to restart the kernel on this Jupyter Notebook**. Only after restarting the Jupyter kernel mlxtend can be used.

With mlxtend installed, we can start by importing the appropriate functions and classes to run Apriori. Here we use `TransactionEncoder`, `apriori`, and `association_rules`. We also load the dataset using pandas and convert it to a list of transactions as we did previously.

In [ ]:
import os
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

store_data = pd.read_csv(os.path.join('Data', 'store_data.csv'), header=None)

transactions = []

# Convert items in the dataframe to string
for row in store_data.values:
    transactions.append([str(item) for item in row if str(item) != 'nan']) # Remove missing values

The `apriori` function expects data in a [one-hot encoded](https://en.wikipedia.org/wiki/One-hot?oldformat=true) Pandas dataframe. In a one-hot enconding, for each unique value in the original categorical column a new column is created. These dummy variables are then filled up with zeros and ones (1 meaning TRUE, 0 meaning FALSE). We can transform the dataset into the right format via the `TransactionEncoder` as follows.

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

Let's convert the enconded transactions into a data frame.

In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

By default, `apriori` returns the column indices of the items. For better readability, we can set parameter `use_colnames=True` to convert these integer values into the respective item names. 

In [ ]:
frequent_itemsets = apriori(df, min_support=0.005, use_colnames=True)
frequent_itemsets

With `frequent_itemset` we can generate association rules by calling the `association_rules` function. This function generates a dataframe including rules and metrics (i.e., score, confidence, and lift).

In [ ]:
association_rules(frequent_itemsets, metric='lift', min_threshold=4)

Leverage (A -> C): Leverage computes the difference between the observed frequency of A and C appearing together and the frequency that would be expected if A and C were independent. An leverage value of 0 indicates independence.

Convinction(A->C): A high conviction value means that the consequent is highly depending on the antecedent. For instance, in the case of a perfect confidence score, the denominator becomes 0 (due to 1 - 1) for which the conviction score is defined as 'inf'. Similar to lift, if items are independent, the conviction is 1.


## 3. Shortcomings of Apriori <a class="anchor" id="shortc"/>

The Apriori algorithm is very useful to find frequent itemsets in transaction data, but it comes with two major shortcomings. The first one refers to the often extremely large set of candidate itemsets generated during the search for frequent itemsets, which consumes a lot of memory. The second one refers to the time spend on counting the support of candidate itemsets since Apriori repeatedly scans the entire database.

These shortcomings are a big problem when (1) we have a large volume of data to be analyzed, and (2) this analysis must be conducted on limited hardware (e.g., a Raspberry Pi).

To overcome these problems, Computer Scientists Jiawei Han, Jian Pei, and Yiwen Yin developed the [FP-Growth](https://en.wikipedia.org/wiki/Association_rule_learning) algorithm in the early 2000s, for which we will look at an implementation next.

## 4. Frequent Pattern Growth <a class="anchor" id="fpg"/>

The big advantage of FP-Growth in comparision to Apriori is that no candidate generation is required. Instead of performing an Apriori-like bottom-up generation of frequent itemset combinations, FP-Growth employs a divide-and-conquer method that only scans the database twice and uses a **frequent pattern tree** (FP-tree) to store data in a more compact way.

The FP-tree is constructed by taking each itemset and mapping it to a path in the tree one at a time. The root starts with a null value, each node represents an item, while the association of the nodes is the itemsets with the order maintained while forming the tree. The whole idea behind this construction is that more frequently occurring itemsets will have a better chance of sharing items.

<div style='align: left; text-align:center;'>
    <img src='img/fpgrowth.png' alt='Example of the FP-Growth algorithm' style="height:500px;"/>
    <span style='display:block;'>Example association rule mining using the FP-Growth algorithm. Image Source: <a href="http://qffc.uic.edu.hk/index.php/content/index/pid/276/cid/6533.html" target="_blank">Quantum Finance Forecast Center</a>.</span>
    <br/>
</div>

### Applying FP-Growth

In the following code blocks, we replace the apriori function with `fpgrowth`. Note that the entire procedure stays unchanged.

In [ ]:
import os
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

store_data = pd.read_csv(os.path.join('Data', 'store_data.csv'), header=None)

transactions = []

# Convert items in the dataframe to string
for row in store_data.values:
    transactions.append([str(item) for item in row if str(item) != 'nan']) # Remove missing values

As in the case of `apriori`, the `fpgrowth` function expects data in a [one-hot encoded](https://en.wikipedia.org/wiki/One-hot?oldformat=true) Pandas data frame.

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

We now convert the enconded transactions into a data frame.

In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

For better readability, we can set parameter `use_colnames=True` to convert these integer values into the respective item names. 

In [ ]:
frequent_itemsets = fpgrowth(df, min_support=0.005, use_colnames=True)
frequent_itemsets

We call the `association_rules` function to generate a data frame of including rules and metrics (i.e., score, confidence, and lift).

In [ ]:
association_rules(frequent_itemsets, metric='lift', min_threshold=4)

## 5. An Alternative FP-Growth implementation <a class="anchor" id="fpg2"/>

As an alternative to mlxtend, we use the [fpgrowth_py](https://github.com/chonyy/fpgrowth_py) package. To install it, you just need to run the command:

```bash
pip install fpgrowth_py
```

Now, let's use the fpgrowth_py to mine association rules.

In [ ]:
import os
import pandas as pd
from fpgrowth_py import fpgrowth

store_data = pd.read_csv(os.path.join('Data', 'store_data.csv'), header=None)

transactions = []

for row in store_data.values:
    transactions.append([str(item) for item in row if str(item) != 'nan']) # Remove missing values

In [ ]:
# minSupRatio is used to compute the support threshold:
# minSup = len(itemSetList) * minSupRatio
freqItemSet, rules = fpgrowth(transactions, minSupRatio=0.005, minConf=0.5)

In [ ]:
for rule in rules:
    # Get items involved in the rule
    print("Rule: ", rule[0], "-> ", rule[1])
    
    # Print the support of the rule
    print("Confidence:", rule[2])
    
    # Print a separator
    print('---')

### A quick comparison between FP-Growth and Apriori

<div style='align: left; text-align:center;'>
    <img src='img/comp_fpgrowth_apriori.png' alt='Comparison between FP-Growth and Apriori' style="height:300px;"/>
    <span style='display:block;'>A comparison between FP-Growth and Apriori. Image Source: <a href="https://towardsdatascience.com/fp-growth-frequent-pattern-generation-in-data-mining-with-python-implementation-244e561ab1c3" target="_blank">Towards Data Science</a>.</span>
    <br/>
</div>


## 6. Exercise: The LastFM Dataset <a class="anchor" id="lastfm"/>

To understand what exactly a listener prefers listening to on the radio, every detail is recorded online. This recorded information is used for recommending music that the listener is likely to enjoy and to come up with a focused marketing strategy that sends out advertisements for music that a listener may wish to buy. However, this results in wasting money on scarce advertising.

Now consider the LastFM dataset extracted from Kaggle (and stored at the Data directory). Suppose that you are provided with data from a music community site, giving you details of each user. This will further help you get access to a log of every artist that the listed users have downloaded on their computer. The objective of providing this data consists of building a system that recommends new artists to the users in this listed community.

How would you apply association rule mining in this case?

# End M2E05: Python Implementations of Association Rule Mining